<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from clickhouse_driver import Client